# Query ODE REST API

ODE REST API's endpoint is at https://oderest.rsl.wustl.edu/live2.
To learn about the parameters the API accepts, there is the manual at http://oderest.rsl.wustl.edu/ODE_REST_V2.1.pdf.

We want to query the API for -- given a ProductID -- where data can be downloaded from.
For that, the query attributes relevant to us are ([ODE REST v2.1 manual, sec.3]):

* `query`: Return list of products with selected product metadata that meet the query parameters
  * `product`
* `productid`: PDS Product Id(s)
* `results`
  * `f`: Product files
  * `p`: PDS Product Identifies
  * `m`: Product metadata
* `output`: query output format, XML or JSON (default XML)
  * `JSON`

And so, our query(ies) should have the following pattern:
```
https://oderest.rsl.wustl.edu/live2?query=product&results=fmp&output=JSON&productid={PRODUCTID}
```
, where `{PRODUCTID}` is a variable for the corresponding ProductID.

[ODE REST v2.1 manual, sec.3]: http://oderest.rsl.wustl.edu/ODE_REST_V2.1.pdf

## Import dataset with product-ids

In [1]:
import pandas as pd

In [2]:
ode_results = 'ODE_Small_SearchResults.CTX.json'

In [3]:
df = pd.read_json(ode_results)
df

,INSTRUMENT_HOST_ID,INSTRUMENT_ID,TYPE,ODE_TYPE_ABBREVIATION,PRODUCT_ID,OBSERVATION_TIME,CENTER_LATITUDE_(IF_AVAILABLE),CENTER_LONGITUDE_(IF_AVAILABLE),ORBITAL_DATA_EXPLORER_PRODUCT_LINK,ORBITAL_DATA_EXPLORER_PRODUCT_FILES_LINK,PDS_VERSION,BUNDLE_ID_(IF_PDS4),COLLECTION_ID_(IF_PDS4),PRODUCT_LID_(IF_PDS4),PRODUCT_TITLE_(IF_PDS4),ORBITAL_DATA_EXPLORER_PRODUCT_DISPLAY_LABEL
353,MRO,CTX,Experiment Data Record,EDR,B18_016496_2076_XN_27N287W,2010-02-01 21:13:31.030,27.3904,72.8253,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,B18_016496_2076_XN_27N287W
354,MRO,CTX,Experiment Data Record,EDR,B19_017208_2095_XN_29N287W,2010-03-29 08:44:56.068,29.3160,72.9383,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,B19_017208_2095_XN_29N287W
355,MRO,CTX,Experiment Data Record,EDR,D02_028179_2089_XI_28N286W,2012-07-31 05:29:04.909,28.6550,73.2590,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,D02_028179_2089_XI_28N286W
356,MRO,CTX,Experiment Data Record,EDR,D04_028812_2082_XI_28N287W,2012-09-18 13:12:21.964,27.9320,72.5210,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,D04_028812_2082_XI_28N287W
357,MRO,CTX,Experiment Data Record,EDR,D16_033282_2088_XI_28N287W,2013-09-01 20:58:25.243,28.5900,72.8440,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,D16_033282_2088_XI_28N287W
358,MRO,CTX,Experiment Data Record,EDR,D16_033348_2088_XI_28N287W,2013-09-07 00:24:27.300,28.5760,72.8470,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,D16_033348_2088_XI_28N287W
359,MRO,CTX,Experiment Data Record,EDR,D17_033928_2089_XI_28N286W,2013-10-22 05:05:23.961,28.6300,73.2540,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,D17_033928_2089_XI_28N286W
360,MRO,CTX,Experiment Data Record,EDR,F22_044504_2092_XN_29N287W,2016-01-24 06:18:37.449,28.9680,72.8140,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,F22_044504_2092_XN_29N287W
361,MRO,CTX,Experiment Data Record,EDR,G02_019120_2088_XN_28N287W,2010-08-25 08:27:28.554,28.5512,72.7657,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,G02_019120_2088_XN_28N287W
362,MRO,CTX,Experiment Data Record,EDR,G21_026557_2089_XN_28N286W,2012-03-26 20:26:23.025,28.6458,73.2371,https://ode.rsl.wustl.edu/mars/indexproductpag...,https://ode.rsl.wustl.edu/mars/productfiles.as...,3,,,,,G21_026557_2089_XN_28N286W


## Define query function and test-query the API

In [4]:
PID = df['PRODUCT_ID'].iloc[0]
PID

'B18_016496_2076_XN_27N287W'

In [5]:
import ode
from ode import request_product

api_endpoint = ode.API_URL

In [6]:
req = request_product(PID, api_endpoint)
req

<Response [200]>

In [7]:
# Response:
# req.json()

### Read Product files of interest

Inspecting this output we recognize the fields of interest as:
* ODEResults
  * Products
    * Product
      * Product_files
        * Product_file: [
          * 'Description' field match 'PRODUCT DATA FILE WITH LABEL'
              * URL
          * 'Description' field match 'BROWSE IMAGE'
              * URL
          * 'Description' field match 'THUMBNAIL IMAGE'
              * URL
        * ]

### Product descriptors

Product files are mapped as:
```
DESCRIPTORS = {
    'product_image': ('Description','PRODUCT DATA FILE WITH LABEL'),
    'browse_image': ('Description','BROWSE IMAGE'),
    'browse_thumbnail': ('Description','THUMBNAIL IMAGE')
}
```

In [8]:
# import importlib
# importlib.reload(ode)

In [9]:
product_files = ode.requested_product_files(req)
product_files

[{'Description': 'PRODUCT DATA FILE WITH LABEL',
  'FileName': 'B18_016496_2076_XN_27N287W.IMG',
  'KBytes': '242001',
  'Type': 'Product',
  'URL': 'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0967/data/B18_016496_2076_XN_27N287W.IMG'},
 {'Description': 'BROWSE IMAGE',
  'FileName': 'B18_016496_2076_XN_27N287W.IMG.JPEG',
  'KBytes': '78',
  'Type': 'Browse',
  'URL': 'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0967/extras/browse/B18_016496_2076_XN_27N287W.IMG.jpeg'},
 {'Description': 'THUMBNAIL IMAGE',
  'FileName': 'B18_016496_2076_XN_27N287W.IMG.JPEG_SMALL',
  'KBytes': '3',
  'Type': 'Browse',
  'URL': 'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0967/extras/thumbnail/B18_016496_2076_XN_27N287W.IMG.jpeg_small'},
 {'Creation_date': '2020-01-16T03:04:07.678',
  'Description': 'PRODUCT KML FILE <A HREF="PAGEHELP/QUICKSTARTGUIDE/ABOUTFOOTPRINTS.HTM" TARGET="_BLANK">HELP</A>',
  'Fi

In [10]:
#importlib.reload(ode)

prod_img_data = ode.find_product_file(product_files, 'product_image', descriptors=ode.DESCRIPTORS['ctx'])
prod_img_browse = ode.find_product_file(product_files, 'browse_image', descriptors=ode.DESCRIPTORS['ctx'])
prod_img_thumbnail = ode.find_product_file(product_files, 'browse_thumbnail', descriptors=ode.DESCRIPTORS['ctx'])

In [11]:
prod_img_data['URL']

'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0967/data/B18_016496_2076_XN_27N287W.IMG'

In [12]:
prod_img_browse['URL']

'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0967/extras/browse/B18_016496_2076_XN_27N287W.IMG.jpeg'

In [13]:
prod_img_thumbnail['URL']

'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0967/extras/thumbnail/B18_016496_2076_XN_27N287W.IMG.jpeg_small'

## Download file(s)

In [14]:
import download
from download import get_product, get_products
from importlib import reload

# Download a file...
download.download_file(prod_img_thumbnail['URL'])

--> Downloading file ./B18_016496_2076_XN_27N287W.IMG.jpeg_small ..
--> File downloaded.


'./B18_016496_2076_XN_27N287W.IMG.jpeg_small'

### Download sample of products (serially)

Downloading:
* product_image : 'PRODUCT DATA FILE WITH LABEL'
* browse_image : 'BROWSE IMAGE'
* browse_thumbnail : 'THUMBNAIL IMAGE'


In [15]:
N_SAMPLE = 2

product_types = ['browse_thumbnail','browse_image']#, 'product_image']

PIDs = df['PRODUCT_ID'].sample(N_SAMPLE)
PIDs

363    G23_027401_2082_XI_28N287W
368    P09_004431_2094_XI_29N287W
Name: PRODUCT_ID, dtype: object

### Get products for `PID`

In [16]:
reload(download)

# file_types: product_types, 
# api_endpoint: ode.API_URL, 
# path: './data/'
kwargs = dict(
    file_types=product_types, 
    descriptors=ode.DESCRIPTORS['ctx'],
    api_endpoint=ode.API_URL, 
    path='./data/')

download.get_product(PID, **kwargs)

Querying ODE for Product B18_016496_2076_XN_27N287W ..
-> Downloading Image data and Browse products for B18_016496_2076_XN_27N287W ..
--> Downloading file ./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg_small ..


./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg_small: 3KB [00:00, 1287.12KB/s]             


--> File downloaded.
--> Downloading file ./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg ..


./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg: 77KB [00:00, 214.95KB/s]                        

--> File downloaded.
-> Data products downloaded.
Done.


{'browse_thumbnail': './data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg_small',
 'browse_image': './data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg'}

### Get Products for `PIDs`

In [17]:
products_data = download.get_products(PIDs, **kwargs)
products_data

Get-Products: (0/2) G23_027401_2082_XI_28N287W
Querying ODE for Product G23_027401_2082_XI_28N287W ..
---
Get-Products: (1/2) P09_004431_2094_XI_29N287W
Querying ODE for Product P09_004431_2094_XI_29N287W ..
-> Downloading Image data and Browse products for P09_004431_2094_XI_29N287W ..
--> Downloading file ./data/P09_004431_2094_XI_29N287W/P09_004431_2094_XI_29N287W.IMG.jpeg_small ..


./data/P09_004431_2094_XI_29N287W/P09_004431_2094_XI_29N287W.IMG.jpeg_small: 1KB [00:00, 373.82KB/s]


--> File downloaded.
--> Downloading file ./data/P09_004431_2094_XI_29N287W/P09_004431_2094_XI_29N287W.IMG.jpeg ..


./data/P09_004431_2094_XI_29N287W/P09_004431_2094_XI_29N287W.IMG.jpeg: 7KB [00:00, 2063.69KB/s]             

--> File downloaded.
-> Data products downloaded.
Done.
---


{'P09_004431_2094_XI_29N287W': {'browse_thumbnail': './data/P09_004431_2094_XI_29N287W/P09_004431_2094_XI_29N287W.IMG.jpeg_small',
  'browse_image': './data/P09_004431_2094_XI_29N287W/P09_004431_2094_XI_29N287W.IMG.jpeg'}}

## Download all Browse products

In [18]:
df['PRODUCT_ID']

353    B18_016496_2076_XN_27N287W
354    B19_017208_2095_XN_29N287W
355    D02_028179_2089_XI_28N286W
356    D04_028812_2082_XI_28N287W
357    D16_033282_2088_XI_28N287W
358    D16_033348_2088_XI_28N287W
359    D17_033928_2089_XI_28N286W
360    F22_044504_2092_XN_29N287W
361    G02_019120_2088_XN_28N287W
362    G21_026557_2089_XN_28N286W
363    G23_027401_2082_XI_28N287W
364    J01_045348_2119_XI_31N287W
365    J02_045493_2115_XI_31N287W
366    P06_003231_2090_XI_29N286W
367    P07_003864_2096_XI_29N286W
368    P09_004431_2094_XI_29N287W
369    P11_005420_2108_XI_30N287W
Name: PRODUCT_ID, dtype: object

In [19]:
# Let's get them all

reload(download)

PIDs = df['PRODUCT_ID']

kwargs.update({'file_types': ['browse_thumbnail','browse_image','product_image']})

Products = download.get_products(PIDs, **kwargs)

Get-Products: (0/17) B18_016496_2076_XN_27N287W
Querying ODE for Product B18_016496_2076_XN_27N287W ..
-> Downloading Image data and Browse products for B18_016496_2076_XN_27N287W ..
--> Downloading file ./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg_small ..


./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg_small: 3KB [00:00, 1175.09KB/s]             


--> File downloaded.
--> Downloading file ./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg ..


./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG.jpeg: 77KB [00:01, 49.30KB/s]                        


--> File downloaded.
--> Downloading file ./data/B18_016496_2076_XN_27N287W/B18_016496_2076_XN_27N287W.IMG ..


...

--> File downloaded.
--> Downloading file ./data/P11_005420_2108_XI_30N287W/P11_005420_2108_XI_30N287W.IMG.jpeg ..


./data/P11_005420_2108_XI_30N287W/P11_005420_2108_XI_30N287W.IMG.jpeg: 53KB [00:00, 207.68KB/s]                        


--> File downloaded.
--> Downloading file ./data/P11_005420_2108_XI_30N287W/P11_005420_2108_XI_30N287W.IMG ..


./data/P11_005420_2108_XI_30N287W/P11_005420_2108_XI_30N287W.IMG: 257861KB [22:12, 193.57KB/s]                             

--> File downloaded.
-> Data products downloaded.
Done.
---
